# Preprocess data for DL models

Simple preprocessing pipeline for the DL models data.

- Load raw EEG data from files;
- Load metadata from files (ages of subjects);
- Apply filters (bandpass);
- Detect potential bad channels and replace them by interpolation;
- Detect potential bad epochs and remove them;
- Save processed data and metadata.

## Import packages & links

In [1]:
# Import packages
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import h5py
import pandas as pd
import mne

In [2]:
from config import PATH_RAW_DATA, PATH_METADATA, PATH_DATA_PROCESSED_DL

## Search all *.cnt files and get paths, code, and age group

In [3]:
import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions

# dir_names = {5: '5mnd mmn',
#              11: '11mnd mmn',
#              17: '17mnd mmn',
#              23: '23mnd mmn',
#              29: '29mnd mmn',
#              35: '35mnd mmn',
#              41: '41mnd mmn',
#              47: '47mnd mmn'}

dir_names = {11: '11mnd mmn',
             17: '17mnd mmn',
             23: '23mnd mmn',
             29: '29mnd mmn',
             35: '35mnd mmn',
             41: '41mnd mmn',
             47: '47mnd mmn'}
    
df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = os.path.join(PATH_RAW_DATA, directory)
    file_names = os.listdir(dir_path)
    cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
    cnt_files = [x.split('/')[-1][:-4] for x in cnt_paths]
    codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    df = pd.DataFrame(list(zip(codes, cnt_paths, cnt_files)), columns =['code', 'cnt_path','cnt_file']) 
    
    df['age_group'] = age_group
    df_list.append(df)

cnt_files = pd.concat(df_list)

In [4]:
cnt_files.head()

,code,cnt_path,cnt_file,age_group
0,1,/Volumes/Seagate Expansion Drive/ePodium/Data/...,001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk,11
1,3,/Volumes/Seagate Expansion Drive/ePodium/Data/...,003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_slp,11
2,4,/Volumes/Seagate Expansion Drive/ePodium/Data/...,004_11_mc_mmn,11
3,5,/Volumes/Seagate Expansion Drive/ePodium/Data/...,005_11_jc_mmn2,11
4,7,/Volumes/Seagate Expansion Drive/ePodium/Data/...,007_11_jc_mmn2_36_wk,11


## Search for all age files and create DataFrame containing all ages per subject code

In [5]:
# age_files = {5: 'ages_5mnths.txt',
#              11: 'ages_11mnths.txt',
#              17: 'ages_17mnths.txt',
#              23: 'ages_23mnths.txt',
#              29: 'ages_29mnths.txt',
#              35: 'ages_35mnths.txt',
#              41: 'ages_41mnths.txt',
#              47: 'ages_47mnths.txt'}

age_files = {11: 'ages_11mnths.txt',
             17: 'ages_17mnths.txt',
             23: 'ages_23mnths.txt',
             29: 'ages_29mnths.txt',
             35: 'ages_35mnths.txt',
             41: 'ages_41mnths.txt',
             47: 'ages_47mnths.txt'}

df_list = []

for age_group, age_file in age_files.items():
    df = pd.read_csv(os.path.join(PATH_METADATA, 'ages', age_file), sep="\t")
    df['age_group'] = age_group
    df_list.append(df)

age_df = pd.concat(df_list)

### Merge the .cnt files with the age information we have on the subjects

In [6]:
merged_df = pd.merge(cnt_files, age_df, how='left', on=['age_group', 'code'])

In [7]:
merged_df.head()

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,1,/Volumes/Seagate Expansion Drive/ePodium/Data/...,001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk,11,329.0,10.966667,0.913889
1,3,/Volumes/Seagate Expansion Drive/ePodium/Data/...,003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_slp,11,336.0,11.200000,0.933333
2,4,/Volumes/Seagate Expansion Drive/ePodium/Data/...,004_11_mc_mmn,11,NaN,NaN,NaN
3,5,/Volumes/Seagate Expansion Drive/ePodium/Data/...,005_11_jc_mmn2,11,329.0,10.966667,0.913889
4,7,/Volumes/Seagate Expansion Drive/ePodium/Data/...,007_11_jc_mmn2_36_wk,11,329.0,10.966667,0.913889


## Helper functions for processing

### Custom cnt-file import function and data standardize function:

In [8]:
def read_cnt_file(file,
                  label_group,
                  event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                  channel_set = "30",
                  tmin = -0.2,
                  tmax = 0.8,
                  lpass = 0.5, 
                  hpass = 40, 
                  threshold = 5, 
                  max_bad_fraction = 0.2,
                  max_bad_channels = 2):
    """ Function to read cnt file. Run bandpass filter. 
    Then detect and correct/remove bad channels and bad epochs.
    Store resulting epochs as arrays.
    
    Args:
    --------
    file: str
        Name of file to import.
    label_group: int
        Unique ID of specific group (must be >0).
    channel_set: str
        Select among pre-defined channel sets. Here: "30" or "62"
    """
    
    if channel_set == "30":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1']
    elif channel_set == "62":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1', 'AFZ', 'PO3', 
                       'P1', 'POZ', 'P2', 'PO4', 'CP2', 'P6', 'M1', 'CP6', 'C6', 'PO8', 'PO7', 
                       'P5', 'CP5', 'CP1', 'C1', 'C2', 'FC2', 'FC6', 'C5', 'FC1', 'F2', 'F6', 
                       'FC5', 'F1', 'AF4', 'AF8', 'F5', 'AF7', 'AF3', 'FPZ']
    else:
        print("Predefined channel set given by 'channel_set' not known...")
        
    
    # Initialize array
    signal_collection = np.zeros((0,len(channel_set),501))
    label_collection = [] #np.zeros((0))
    channel_names_collection = []
    
    # Import file
    try:
        data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
    except ValueError:
        print("ValueError")
        print("Could not load file:", file)
        return None, None, None
    
    # Band-pass filter (between 0.5 and 40 Hz. was 0.5 to 30Hz in Stober 2016)
    data_raw.filter(0.5, 40, fir_design='firwin')

    # Get events from annotations in the data
    events_from_annot, event_dict = mne.events_from_annotations(data_raw)
    
    # Set baseline:
    baseline = (None, 0)  # means from the first instant to t = 0

    # Select channels to exclude (if any)
    channels_exclude = [x for x in data_raw.ch_names if x not in channel_set]
    channels_exclude = [x for x in channels_exclude if x not in ['HEOG', 'VEOG']]
    
    for event_id in event_idx:
        if str(event_id) in event_dict:
            # Pick EEG channels
            picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                               #exclude=data_exclude)#'bads'])
                                   include=channel_set, exclude=channels_exclude)#'bads'])

            epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                baseline=baseline, preload=True, event_repeated='merge', verbose=False)

            # Detect potential bad channels and epochs
            bad_channels, bad_epochs = helper_functions.select_bad_epochs(epochs,
                                                                          event_id,
                                                                          threshold = threshold,
                                                                          max_bad_fraction = max_bad_fraction)

            # Interpolate bad channels
            # ------------------------------------------------------------------
            if len(bad_channels) > 0:
                if len(bad_channels) > max_bad_channels:
                    print(20*'--')
                    print("Found too many bad channels (" + str(len(bad_channels)) + ")")
                    return None, None, None
                else:
                    montage = mne.channels.make_standard_montage('standard_1020')
                    montage.ch_names = [ch_name.upper() for ch_name in montage.ch_names]
                    data_raw.set_montage(montage)
                    
                    # MARK: Think about using all channels before removing (62 -> 30), to enable for better interpolation
                    
                    # Mark bad channels:
                    data_raw.info['bads'] = bad_channels
                    # Pick EEG channels:
                    picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                                       #exclude=data_exclude)#'bads'])
                                       include=channel_set, exclude=channels_exclude)#'bads'])
                    epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                        tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                        baseline=baseline, preload=True, verbose=False)
                    
                    # Interpolate bad channels using functionality of 'mne'
                    epochs.interpolate_bads()
                    

            # Get signals as array and add to total collection
            channel_names_collection.append(epochs.ch_names)
            signals_cleaned = epochs[str(event_id)].drop(bad_epochs).get_data()
            signal_collection = np.concatenate((signal_collection, signals_cleaned), axis=0)
            label_collection += [event_id + label_group] * signals_cleaned.shape[0]

    return signal_collection, label_collection, channel_names_collection

In [9]:
"""Functions to import and process EEG data from cnt files.
"""
def standardize_EEG(data_array,
                    std_aim = 1,
                    centering = 'per_channel',
                    scaling = 'global'):
    """ Center data around 0 and adjust standard deviation.

    Args:
    --------
    data_array: np.ndarray
        Input data.
    std_aim: float/int
        Target standard deviation for rescaling/normalization.
    centering: str
        Specify if centering should be done "per_channel", or "global".
    scaling: str
        Specify if scaling should be done "per_channel", or "global".
    """
    if centering == 'global':
        data_mean = data_array.mean()

        # Center around 0
        data_array = data_array - data_mean

    elif centering == 'per_channel':
        for i in range(data_array.shape[1]):

            data_mean = data_array[:,i,:].mean()

            # Center around 0
            data_array[:,i,:] = data_array[:,i,:] - data_mean

    else:
        print("Centering method not known.")
        return None

    if scaling == 'global':
        data_std = data_array.std()

        # Adjust std to std_aim
        data_array = data_array * std_aim/data_std

    elif scaling == 'per_channel':
        for i in range(data_array.shape[1]):

            data_std = data_array[:,i,:].std()

            # Adjust std to std_aim
            data_array[:,i,:] = data_array[:,i,:] * std_aim/data_std
    else:
        print("Given method is not known.")
        return None

    return data_array

# Preprocessing raw

In [14]:
import re
from itertools import islice

for i, file in islice(merged_df.iterrows(), 0, None): # Enables you to set starting row of DataFrame
    print(f"Checking out file: {file['cnt_file']}")
        
    filename_processed_raw = os.path.join(PATH_DATA_PROCESSED_DL, "processed_raw" + file['cnt_file'] + ".npy")
    filename_metadata = os.path.join(PATH_DATA_PROCESSED_DL, "processed_metadata_" + file['cnt_file'] + ".csv")
    
    # Step 1: Check if file don't already exist, else skip
    if os.path.exists(filename_processed_raw) and os.path.exists(filename_metadata):
        print(f"Skipping because {filename_processed_raw} and {filename_metadata} already exist.")
        continue
    
    
    # Step 2: Import data and labels 
    signal_collect, label_collect, ch_names = read_cnt_file(file['cnt_path'],
                                                            file['age_months'],
                                                            event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                                                            channel_set = "30",
                                                            tmin = -0.2,
                                                            tmax = 0.8,
                                                            lpass = 0.5, 
                                                            hpass = 40, 
                                                            threshold = 5, 
                                                            max_bad_fraction = 0.2,
                                                            max_bad_channels = 2)    
    
    if signal_collect is None:
        continue
        
    # Step 3: standardize data
    if signal_collect is not None:
        signal_collect = standardize_EEG(signal_collect,
                                         std_aim = 1,
                                         centering = 'per_channel',
                                         scaling = 'global')

    # Step 4: Save raw data and metadata
    if signal_collect is not None:
        np.save(filename_processed_raw, signal_collect)
        pd.DataFrame(file).transpose().to_csv(filename_metadata, sep=',', index=False, header=True)     
    
    # TODO: 1. Try saving 2. Retry after n seconds if fails 3. If fails m times, continue

Checking out file: 001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk
Used Annotations descriptions: ['2', '3', '4', '5', '55', '66', '77', '88']
Found 247 bad epochs in a total of 30  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Marked 129 bad epochs in a total of 360  epochs.
Found 220 bad epochs in a total of 27  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Marked 66 bad epochs in a total of 360  epochs.
Found 311 bad epochs in a total of 30  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  8
Marked 200 bad epochs in a total of 360  epochs.
----------------------------------------
Found too many bad channels (3)
Checking out file: 003_11_jc_mmn36_slp_mmn25_wk_mmn4

Checking out file: 014_11_mc_mmn36_wk_mmn25_wk
Used Annotations descriptions: ['0', '12', '13', '2', '3', '55', '66']
Found 284 bad epochs in a total of 30  channels.
Found bad channel (more than 57.6  bad epochs): Channel no:  19
Found bad channel (more than 57.6  bad epochs): Channel no:  6
Found bad channel (more than 57.6  bad epochs): Channel no:  8
Found bad channel (more than 57.6  bad epochs): Channel no:  9
Marked 100 bad epochs in a total of 288  epochs.
----------------------------------------
Found too many bad channels (4)
Checking out file: 015_11_jc_mmn36_slp_mmn25_slp_mmn47_slp_mmn58_slp
Used Annotations descriptions: ['12', '13', '14', '15', '2', '3', '4', '5', '55', '66', '77', '88']
Found 16 bad epochs in a total of 2  channels.
Marked 16 bad epochs in a total of 400  epochs.
Found 23 bad epochs in a total of 4  channels.
Marked 23 bad epochs in a total of 400  epochs.
Found 44 bad epochs in a total of 8  channels.
Marked 44 bad epochs in a total of 400  epochs.
Foun

Found 45 bad epochs in a total of 13  channels.
Marked 45 bad epochs in a total of 366  epochs.
Found 4 bad epochs in a total of 1  channels.
Marked 4 bad epochs in a total of 50  epochs.
No outliers found with given threshold.
Found 6 bad epochs in a total of 5  channels.
Marked 6 bad epochs in a total of 44  epochs.
Checking out file: 030_11_jc_mmn25_wk
Used Annotations descriptions: ['0', '12', '2', '55']
Found 281 bad epochs in a total of 29  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  17
Marked 116 bad epochs in a total of 400  epochs.
Found 31 bad epochs in a total of 12  channels.
Found bad channel (more than 10.0  bad epochs): Channel no:  17
Marked 10 bad epochs in a total of 50  epochs.
Checking out file: 030_11_jc_mmn36_wk_mmn25_wk
Used Annotations descriptions: ['0', '13', '3', '66']
Found 33 bad epochs in a total of 28  channels.
Marked 33 bad epochs in a total of 400  epochs.
Found 5 bad epochs in a total of 27  channels.
Marked 5 bad epochs in 

Used Annotations descriptions: ['4', '5', '77', '88']
Found 454 bad epochs in a total of 30  channels.
Found bad channel (more than 91.0  bad epochs): Channel no:  0
Found bad channel (more than 91.0  bad epochs): Channel no:  6
Found bad channel (more than 91.0  bad epochs): Channel no:  12
Found bad channel (more than 91.0  bad epochs): Channel no:  14
Found bad channel (more than 91.0  bad epochs): Channel no:  1
Found bad channel (more than 91.0  bad epochs): Channel no:  11
Found bad channel (more than 91.0  bad epochs): Channel no:  18
Found bad channel (more than 91.0  bad epochs): Channel no:  4
Found bad channel (more than 91.0  bad epochs): Channel no:  2
Found bad channel (more than 91.0  bad epochs): Channel no:  3
Found bad channel (more than 91.0  bad epochs): Channel no:  24
Found bad channel (more than 91.0  bad epochs): Channel no:  25
Found bad channel (more than 91.0  bad epochs): Channel no:  22
Found bad channel (more than 91.0  bad epochs): Channel no:  27
Found b

Used Annotations descriptions: ['0', '1', '12', '13', '14', '15', '18', '2', '3', '4', '48', '49', '5', '55', '66', '77', '80', '81', '88']
Found 103 bad epochs in a total of 29  channels.
Marked 103 bad epochs in a total of 400  epochs.
Found 289 bad epochs in a total of 30  channels.
Marked 289 bad epochs in a total of 801  epochs.
Found 52 bad epochs in a total of 8  channels.
Marked 52 bad epochs in a total of 401  epochs.
Found 31 bad epochs in a total of 10  channels.
Marked 31 bad epochs in a total of 401  epochs.
Found 12 bad epochs in a total of 12  channels.
Marked 12 bad epochs in a total of 50  epochs.
Found 33 bad epochs in a total of 22  channels.
Marked 33 bad epochs in a total of 100  epochs.
Found 7 bad epochs in a total of 3  channels.
Marked 7 bad epochs in a total of 50  epochs.
Found 5 bad epochs in a total of 5  channels.
Marked 5 bad epochs in a total of 50  epochs.
Checking out file: 122_11_mr_mmn36_wk_mmn25_wk_mmn47_wk
Used Annotations descriptions: ['12', '13'

...

Used Annotations descriptions: ['13', '3', '66']
Found 8 bad epochs in a total of 5  channels.
Marked 8 bad epochs in a total of 580  epochs.
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 75  epochs.
Checking out file: 605-131-41m-jc-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 14 bad epochs in a total of 12  channels.
Marked 14 bad epochs in a total of 800  epochs.
No outliers found with given threshold.
Checking out file: 610-185-41m-jc-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 30 bad epochs in a total of 30  channels.
Marked 30 bad epochs in a total of 735  epochs.
Found 9 bad epochs in a total of 9  channels.
Marked 9 bad epochs in a total of 89  epochs.
Checking out file: 611-157-41m-mc-mmn39
Used Annotations descriptions: ['0', '13', '3', '99']
Found 16 bad epochs in a total of 30  channels.
Marked 16 bad epochs in a total of 212  epochs.
No outliers found with given threshold.
Checking out file: 611-157-41m-mc-

Checking out file: 701-008-41m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 17 bad epochs in a total of 6  channels.
Marked 17 bad epochs in a total of 800  epochs.
No outliers found with given threshold.
Checking out file: 702-071-41m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 80 bad epochs in a total of 29  channels.
Marked 80 bad epochs in a total of 875  epochs.
Found 21 bad epochs in a total of 9  channels.
Marked 21 bad epochs in a total of 109  epochs.
Checking out file: 703-026-41m-mr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 19 bad epochs in a total of 8  channels.
Marked 19 bad epochs in a total of 800  epochs.
Found 3 bad epochs in a total of 3  channels.
Marked 3 bad epochs in a total of 100  epochs.
Checking out file: 704-032-41m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 103 bad epochs in a total of 20  channels.
Marked 103 bad epochs in a total of 800  epochs.
Found 16 bad epochs in a total o

Found 84 bad epochs in a total of 6  channels.
Marked 84 bad epochs in a total of 800  epochs.
Found 10 bad epochs in a total of 28  channels.
Marked 10 bad epochs in a total of 100  epochs.
Checking out file: 730-201-41m-jr-mmn
Used Annotations descriptions: ['0', '13', '3', '66']
Found 46 bad epochs in a total of 12  channels.
Marked 46 bad epochs in a total of 400  epochs.
Found 4 bad epochs in a total of 1  channels.
Marked 4 bad epochs in a total of 50  epochs.
Checking out file: 730-201-41m-jr-mmn36
Used Annotations descriptions: ['0', '13', '3', '66']
Found 9 bad epochs in a total of 29  channels.
Marked 9 bad epochs in a total of 400  epochs.
No outliers found with given threshold.
Checking out file: 731-197-41m-jr-mmn36
Used Annotations descriptions: ['13', '3', '66']
Found 8 bad epochs in a total of 3  channels.
Marked 8 bad epochs in a total of 800  epochs.
No outliers found with given threshold.
Checking out file: 732-303-41m-mr-mmn36
Used Annotations descriptions: ['13', '

Used Annotations descriptions: ['13', '3', '99']
Found 10 bad epochs in a total of 3  channels.
Marked 10 bad epochs in a total of 1200  epochs.
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad epochs in a total of 300  epochs.
Checking out file: 755-471-41m-mr-mmn39(langzaam)
Used Annotations descriptions: ['13', '3', '99']
Found 38 bad epochs in a total of 30  channels.
Marked 38 bad epochs in a total of 600  epochs.
Found 9 bad epochs in a total of 30  channels.
Marked 9 bad epochs in a total of 150  epochs.
Checking out file: 756-467-41m-mr-mmn39(snel)
Used Annotations descriptions: ['13', '3', '99']
Found 94 bad epochs in a total of 14  channels.
Marked 94 bad epochs in a total of 1193  epochs.
Found 19 bad epochs in a total of 11  channels.
Marked 19 bad epochs in a total of 298  epochs.
Checking out file: 001_47_jc_mmn25_slp
Used Annotations descriptions: ['12', '2', '55']
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 400  epochs.
Foun

Found 34 bad epochs in a total of 8  channels.
Marked 34 bad epochs in a total of 800  epochs.
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad epochs in a total of 100  epochs.
Checking out file: 406_47_md_mmn36_wk
Used Annotations descriptions: ['13', '3', '66']
Found 23 bad epochs in a total of 30  channels.
Marked 23 bad epochs in a total of 800  epochs.
Found 1 bad epochs in a total of 30  channels.
Marked 1 bad epochs in a total of 100  epochs.
Checking out file: 407_47_jd_mmn25_wk
Used Annotations descriptions: ['12', '2', '55']
Found 79 bad epochs in a total of 16  channels.
Marked 79 bad epochs in a total of 800  epochs.
Found 9 bad epochs in a total of 6  channels.
Marked 9 bad epochs in a total of 100  epochs.
Checking out file: 407_47_jd_mmn36_wk
Used Annotations descriptions: ['13', '3', '66']
Found 83 bad epochs in a total of 17  channels.
Marked 83 bad epochs in a total of 800  epochs.
Found 12 bad epochs in a total of 10  channels.
Marked 12 bad epochs in a to

## Transform .npy to .zarr

In [11]:
%%time

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_DL)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(file_name)[0] for file_name in fnmatch.filter(file_names, "*.npy")]

for file in files:
    # Step 0: Get the NumPy file name
    np_name = file + ".npy"

    # Step 1: Load existing NumPy array
    X = np.load(os.path.join(PATH_DATA_PROCESSED_DL, np_name))

    # Step 2: Open a Zarr file with the same name and assign the number of epochs as chunks to the file
    zarr_name = np_name.replace(".npy", ".zarr")
    z_file =  zarr.open(os.path.join(PATH_DATA_PROCESSED_DL, zarr_name), 
                        mode='w', 
                        shape=X.shape, 
                        chunks=(1, X.shape[1], X.shape[2]))
    z_file[:] = X

## TODO: Remove files that have len(npy array) == 0